In [1]:
%pylab inline 
import pandas as pd 
import json

Populating the interactive namespace from numpy and matplotlib


In [2]:
df_data = []
with open('../system/user_study_2018_03_09.log') as f:
    for line in f:
        lsp = line.split(',')
        date = lsp[0]
        user = lsp[1][1:-1]
        task = lsp[2][1:-1]
        dataset_name = lsp[3][1:-1]
        clicks = json.loads(",".join(lsp[4:]))
        if dataset_name!="default":
            dataset = json.loads(open("../system/static/generated_dashboards/"+dataset_name).read().replace("\\",""))
            for nodeID in dataset.keys():
                df_data.append([date,dataset_name,user,task,nodeID,clicks[nodeID],dataset[nodeID][-1]["filter"],dataset[nodeID][-1]["populationSize"]])

In [3]:
df = pd.DataFrame(df_data,columns=["date","dataset_name","user","task","nodeID","clicks","visualization","populationSize"])
df["date"] = pd.to_datetime(df["date"],format="%Y-%m-%d-%H-%M-%S")

In [4]:
df.loc[df["clicks"]==0,"interested"]="unselected"
df.loc[df["clicks"]==1,"interested"]="unselected"
df.loc[df["clicks"]==2,"interested"]="interested"
df.loc[df["clicks"]==3,"interested"]="not interested"

In [5]:
#select the best task, user (since all node are logged on the latest output, so we will just pick the latest one)
df = df.loc[df.groupby(["user","task","nodeID"])["date"].idxmax()]

In [6]:
df = df[["user","visualization","interested","task"]]

In [7]:
def cleanFilterVal(x):
    if x=="#":
        return "overall"
    else:
        return x.replace("$","=").replace("#",", ")[1:-2]

In [8]:
df["visualization"]= df["visualization"].apply(cleanFilterVal)

In [9]:
df.to_csv("user_click_data.csv",index=None)

In [10]:
df 

,user,visualization,interested,task
477,1,overall,unselected,A1
474,1,"search_conducted=f, duration=30+ min, stop_ti...",unselected,A1
471,1,"search_conducted=t, duration=30+ min, stop_ti...",unselected,A1
470,1,duration=30+ min,not interested,A1
479,1,"search_conducted=f, duration=30+ min",interested,A1
473,1,"search_conducted=t, duration=30+ min",interested,A1
476,1,"duration=30+ min, stop_time=Late Evening",not interested,A1
475,1,"duration=30+ min, stop_time=Morning",not interested,A1
478,1,"search_conducted=t, contraband_found=f, durat...",unselected,A1
472,1,"driver_race=White, duration=30+ min, stop_tim...",unselected,A1
